In [ ]:
from google.colab import drive
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = ['Times New Roman'] + plt.rcParams['font.serif']
from scipy.special import gamma
import matplotlib.animation as manimation
drive.mount("/content/gdrive")
import os
!pip install gplearn
from gplearn.genetic import SymbolicRegressor
from sklearn.model_selection import train_test_split
from sympy import *
import pandas as pd
from tqdm.notebook import tqdm

In [ ]:
V_hist = np.load('/content/gdrive/My Drive/V_hist.npy', allow_pickle=True)
psi_hist = np.load('/content/gdrive/My Drive/psi_hist.npy', allow_pickle=True)
t_hist = np.load('/content/gdrive/My Drive/t_hist_qs.npy', allow_pickle=True)

In [ ]:
N = 512
dt = 0.0001

In [ ]:
def getGradient(f, dx):

  R = -1  
  L = 1   
  
  f_dx = ( np.roll(f,R,axis=0) - np.roll(f,L,axis=0) ) / (2*dx)
  f_dy = ( np.roll(f,R,axis=1) - np.roll(f,L,axis=1) ) / (2*dx)
  
  return f_dx, f_dy

In [ ]:
def getLaplacian(f, N):

  dx = 1/N

  R = -1   
  L = 1    
  
  f_dx = ( np.roll(f,R,axis=0) - np.roll(f,L,axis=0) ) / (2*dx)
  f_dy = ( np.roll(f,R,axis=1) - np.roll(f,L,axis=1) ) / (2*dx)

  f_dx_dx = ( np.roll(f_dx,R,axis=0) - np.roll(f_dx,L,axis=0) ) / (2*dx)
  f_dy_dy = ( np.roll(f_dy,R,axis=1) - np.roll(f_dy,L,axis=1) ) / (2*dx)

  laplacian = f_dx_dx + f_dy_dy
  
  return laplacian

In [ ]:
def getTimeDerivative(i, f_hist, dt):

  f_dt = (f_hist[i + 1] - f_hist[i - 1])/(2*dt)

  return f_dt

In [ ]:
N = 512
dt = 0.0001
length = len(psi_hist)

In [ ]:
for i in tqdm(range(length)):

  if i == 0 or i == (length - 1):
    continue

  V = V_hist[i]
  V_dt = (V_hist[i + 1] - V_hist[i - 1])/(2*dt)
  V_laplace = getLaplacian(V, N)
  V_dx, V_dy = getGradient(V,N)

  psi = np.real(psi_hist[i])
  psi_dt = np.real((psi_hist[i + 1] - psi_hist[i - 1])/(2*dt) * 1j)
  psi_laplace = np.real(getLaplacian(psi, N))
  psi_dx, psi_dy = np.real(getGradient(psi,N))

  if i == 1:

    df = pd.DataFrame()

    df['psi_dt'] = np.ravel(psi_dt)
    df['psi_laplace'] = np.ravel(psi_laplace)
    df['psi_dx'] = np.ravel(psi_dx)
    df['psi_dy'] = np.ravel(psi_dy)
    df['psi'] = np.ravel(psi)

    df['V_dt'] = np.ravel(V_dt)
    df['V_laplace'] = np.ravel(V_laplace)
    df['V_dx'] = np.ravel(V_dx)
    df['V_dy'] = np.ravel(V_dy)
    df['V'] = np.ravel(V)

  else:

    dfa = pd.DataFrame()
    
    dfa['psi_dt'] = np.ravel(psi_dt)
    dfa['psi_laplace'] = np.ravel(psi_laplace)
    dfa['psi_dx'] = np.ravel(psi_dx)
    dfa['psi_dy'] = np.ravel(psi_dy)
    dfa['psi'] = np.ravel(psi)

    dfa['V_dt'] = np.ravel(V_dt)
    dfa['V_laplace'] = np.ravel(V_laplace)
    dfa['V_dx'] = np.ravel(V_dx)
    dfa['V_dy'] = np.ravel(V_dy)
    dfa['V'] = np.ravel(V)

    df = pd.concat([df, dfa], axis=0)

In [ ]:
y = df.iloc[:120000,0]
X = df.iloc[:120000, 1:]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [ ]:
converter = {
    'sub': lambda x, y : x - y,
    'div': lambda x, y : x/y,
    'mul': lambda x, y : x*y,
    'add': lambda x, y : x + y,
    'neg': lambda x    : -x,
    'pow': lambda x, y : x**y,
    'sin': lambda x    : sin(x),
    'cos': lambda x    : cos(x),
    'inv': lambda x: 1/x,
    'sqrt': lambda x: x**0.5,
    'tan': lambda x: tan(x),
    'log': lambda x: log(x)

}

In [ ]:
#expanded variable set

for i in range(21):


  # First Test
  function_set = ['add', 'sub', 'mul']
  est_gp = SymbolicRegressor(population_size=1000,function_set=function_set,
                             generations=50, stopping_criteria=0.01,
                             p_crossover=0.7, p_subtree_mutation=0.1,
                             p_hoist_mutation=0.05, p_point_mutation=0.1,
                             max_samples=0.9, verbose=1,
                             parsimony_coefficient=0.025, random_state=i,
                            feature_names=X_train.columns)

  est_gp.fit(X_train, y_train)
  print()

  print('R2:',est_gp.score(X_test,y_test))
  print()
  next_e = sympify(str((est_gp._program)), locals=converter)
  print(next_e)
  print()
  print()

In [ ]:
#expanded function set and expanded variable set

for i in range(42):


  # First Test
  function_set = ['add', 'sub', 'mul', 'div', 'neg', 'inv', 'sqrt', 'sin', 'cos', 'tan', 'log', 'max', 'min']
  est_gp = SymbolicRegressor(population_size=1000,function_set=function_set,
                             generations=50, stopping_criteria=0.01,
                             p_crossover=0.7, p_subtree_mutation=0.1,
                             p_hoist_mutation=0.05, p_point_mutation=0.1,
                             max_samples=0.9, verbose=1,
                             parsimony_coefficient=0.025, random_state=i,
                            feature_names=X_train.columns)

  est_gp.fit(X_train, y_train)
  print()

  print('R2:',est_gp.score(X_test,y_test))
  print()
  next_e = sympify(str((est_gp._program)), locals=converter)
  print(next_e)
  print()
  print()